In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from dataclasses import dataclass
from typing import *
import logging

import lass.train
import lass.datasets
import lass.pipeline
from lass.log_handling import LogLoader, LogLoaderArgs, PaperTasks

# Construct a set of tasks with the same number of examples across all shots.

In [3]:
log_loader_args = LogLoaderArgs(
    logdir='../artifacts/logs/',
    tasks='paper-full',
    model_families=['BIG-G T=0'],
    model_sizes=['128b'],
    shots=[0, 1, 2, 3],
    query_types=['multiple_choice'],
)
log_loader = LogLoader(log_loader_args)

logging.info("Starting data loading")
data = lass.datasets.to_dataframe(log_loader)
logging.info("Loaded data.")

data = lass.pipeline.binarize(data)
data = lass.pipeline.augment(data)
data = lass.pipeline.clean(data)

uniqueness = data.groupby(['task', 'shots']).size().groupby('task').agg(lambda x: x.nunique() == 1)
filtered_tasks = uniqueness[uniqueness == True].index.tolist()
print(f"#Filtered tasks: {len(filtered_tasks)}. From total of {len(uniqueness)}, {len(uniqueness) - len(filtered_tasks)} tasks have different number of examples across shots.")

/home/wout/pp/lass/src/lass/pipeline.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'correct'] = df['correct'].astype(int)


#Filtered tasks: 114. From total of 118, 4 tasks have different number of examples across shots.


# Train model for each shot.

In [4]:
@dataclass
class Architecture():
    name: str
    name_short: str
    batch_size: int
    gradient_accumulation_steps: int
    
architecture = Architecture(
    name="microsoft/deberta-v3-base", 
    name_short="deberta-base",
    batch_size=16,
    gradient_accumulation_steps=2,
)


# for n_shots in [0, 1, 2, 3]:
for n_shots in [2, 3]:
    print(f"#Examples for {n_shots} shots: {len(data[data.shots == n_shots])}")
    shot_log_loader_args = LogLoaderArgs(**(log_loader_args.__dict__ | {
        'shots': [n_shots],
        'tasks': filtered_tasks,
    }))

    def t(truncation_side: Union[Literal["left"], Literal["right"]]):
        lass.train.train(
            data_args=shot_log_loader_args,
            group='shot-influence',
            output_dir="shot-influence",
            split='instance',
            model_name=architecture.name,
            model_name_short=f"{architecture.name_short}-for-{n_shots}-shots-t-{truncation_side}",
            truncation_side=truncation_side,
            batch_size=architecture.batch_size,
            gradient_accumulation_steps=architecture.gradient_accumulation_steps,
            include_model_in_input=False,
            include_n_targets_in_input=False,
            n_epochs=6,
            extra_training_args={
                "warmup_steps": 3000,
                "learning_rate": 2e-5,
            },
        )

    t('right')
    t('left')

#Examples for 2 shots: 54085


{'text': 'In what follows, we provide short narratives, each of which illustrates a common proverb. \nNarrative: The anxiety about getting the injection made her unable to sleep that night, but the actual experience wasn\'t as bad as she thought it would be.\nThis narrative is a good illustration of the following proverb: Don\'t meet troubles half-way\n\nNarrative: Inmate 76853 lay on the execution gurney ready to utter his last words before lethal drugs ended his life.  He looked through the mirror into the witness room and said to his family and his victim\'s family: "I killed Victoria.  It was wrong.  I can\'t take it back.  I earned my punishment, and I accept it."  Inmate 76853 turned his head back to stare at the stark ceiling and paid his debt with his life.\nThis narrative is a good illustration of the following proverb: The wages of sin is death\n\nNarrative: A father brought his son to the local airport for a show. They got up close to an plane in the show. The pilot arrives 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/home/wout/pp/lass/.env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/43 [00:00<?, ?ba/s]

...

Model weights saved in shot-influence/deberta-base-for-3-shots-t-left-bs16*2-3sh-instance-split-12150358/checkpoint-7500/pytorch_model.bin


Deleting older checkpoint [shot-influence/deberta-base-for-3-shots-t-left-bs16*2-3sh-instance-split-12150358/checkpoint-7000] due to args.save_total_limit




Training completed. Do not forget to share your model on huggingface.co/models =)




Loading best model from shot-influence/deberta-base-for-3-shots-t-left-bs16*2-3sh-instance-split-12150358/checkpoint-5000 (score: 0.5734668374061584).


eval/accuracy,▁▄▇▆▆▆▅▄▆▇▇▇█▆▆
eval/balanced_accuracy,▁█▇▇▆█▆▃▆▆██▇▇▇
eval/bs,▄▃▂▃▂▃▁▃▂▁▄▄▃██
eval/bs_dsc,▁▄▅▅▆▄▅▆▅█▆█▇▃▄
eval/bs_mcb,▂▁▁▂▂▂▁▃▂▂▄▅▄▇█
eval/bs_unc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_balanced_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_bs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_bs_dsc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_bs_mcb,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
